In [65]:
import h5py 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pydicom
from glob import glob
import cv2
from matplotlib import animation
from IPython import display
import torch
from torch.nn import functional as F
from tqdm.auto import trange
from sklearn.model_selection import GroupKFold
import albumentations as A
from albumentations.pytorch import ToTensorV2
import random

In [66]:
train_descriptions = pd.read_csv("../../input/rsna-2024-lumbar-spine-degenerative-classification/train_series_descriptions.csv")
train_coord_df = pd.read_csv("../../input/rsna-2024-lumbar-spine-degenerative-classification/train_label_coordinates.csv")
train_coord_df = train_coord_df.merge(train_descriptions, on=["series_id", "study_id"], how="left")
train_df = pd.read_csv("../../input/rsna-2024-lumbar-spine-degenerative-classification/train.csv")
train_df["fold_id"] = -1
for i, (train_index, valid_index) in enumerate(GroupKFold(n_splits=5).split(train_df, np.arange(len(train_df)), train_df.study_id)):
    train_df.loc[valid_index, "fold_id"] = i
train_coord_df = train_coord_df.merge(train_df.loc[:, ["study_id", "fold_id"]], on=["study_id"], how="left")
train_coord_df = train_coord_df.sort_values(by=["study_id", "series_id", "level", "condition"]).reset_index(drop=True)

In [67]:
idx = 0

In [68]:
coord_label_num = train_coord_df.groupby("series_id").count().sort_values("series_id").study_id.to_numpy()
SCS = train_coord_df.groupby("series_id").head(1).sort_values("series_id").condition.to_numpy() 
series_id =  train_coord_df.groupby("series_id").head(1).sort_values("series_id").series_id
new_series_id = series_id[((SCS == "Spinal Canal Stenosis") & (coord_label_num == 5)) | ((SCS != "Spinal Canal Stenosis") & (coord_label_num == 10))]
new_train_coord_df = train_coord_df[train_coord_df.series_id.isin(new_series_id)].sort_values(by=["series_id", "level"]).reset_index(drop=True)
series_ids = new_train_coord_df.series_id.unique()
series_df = new_train_coord_df[new_train_coord_df.series_id == series_ids[idx]]
idx += 1
series_df

,study_id,series_id,instance_number,condition,level,x,y,series_description,fold_id
0,3996069892,10996,13,Left Neural Foraminal Narrowing,L1/L2,266.827697,166.544283,Sagittal T1,0
1,3996069892,10996,6,Right Neural Foraminal Narrowing,L1/L2,264.630404,163.686737,Sagittal T1,0
2,3996069892,10996,12,Left Neural Foraminal Narrowing,L2/L3,252.811594,205.294686,Sagittal T1,0
3,3996069892,10996,6,Right Neural Foraminal Narrowing,L2/L3,258.064459,197.391921,Sagittal T1,0
4,3996069892,10996,12,Left Neural Foraminal Narrowing,L3/L4,249.513688,243.220612,Sagittal T1,0
5,3996069892,10996,5,Right Neural Foraminal Narrowing,L3/L4,250.185325,238.976237,Sagittal T1,0
6,3996069892,10996,13,Left Neural Foraminal Narrowing,L4/L5,249.513688,282.795491,Sagittal T1,0
7,3996069892,10996,4,Right Neural Foraminal Narrowing,L4/L5,249.747596,286.251040,Sagittal T1,0
8,3996069892,10996,13,Left Neural Foraminal Narrowing,L5/S1,251.987118,319.896940,Sagittal T1,0
9,3996069892,10996,5,Right Neural Foraminal Narrowing,L5/S1,247.558947,321.269412,Sagittal T1,0


In [47]:
h5f = h5py.File("../../input/resize_volume_256x256.h5")
volume = h5f[str(series_df.series_id.iloc[0])][:]

# fig = plt.figure(figsize=(6, 6))
# im = plt.imshow(volume[0], cmap="gray")
# def draw(i):
#     im.set_array(volume[i])
#     return [im]
# anim = animation.FuncAnimation(
#     fig, draw, frames=volume.shape[0], interval=200, blit=True
# )
# plt.close()
# display.HTML(anim.to_jshtml())

In [48]:
volume = torch.tensor(volume)
print(volume.shape)
step = 2 #if series_df.series_description.iloc[0] == "Axial T2" else 1
volume = volume.unfold(0, size=3, step=step).numpy()
print(volume.shape)
# fig = plt.figure(figsize=(6, 6))
# im = plt.imshow(volume_torch[0].permute(1, 2, 0))
# def draw(i):
#     im.set_array(volume_torch[i].permute(1, 2, 0))
#     return [im]
# anim = animation.FuncAnimation(
#     fig, draw, frames=volume_torch.shape[0], interval=200, blit=True
# )
# plt.close()
# display.HTML(anim.to_jshtml())

torch.Size([20, 256, 256])
(9, 256, 256, 3)


In [49]:
if len(series_df) == 10:
    new_series_df = series_df.iloc[::2].reset_index(drop=True)
    new_series_df.loc[:, ["instance_number", "x", "y"]] = \
        (series_df.iloc[0::2].loc[:, ["instance_number", "x", "y"]].to_numpy().astype(np.float32) \
         + series_df.iloc[1::2].loc[:, ["instance_number", "x", "y"]].to_numpy().astype(np.float32)) / 2
else:
    assert len(series_df) == 5
    new_series_df = series_df.reset_index(drop=True)
img_paths = glob(f"../../input/rsna-2024-lumbar-spine-degenerative-classification/train_images/**/{series_df.series_id.iloc[0]}/*.dcm")
dicom = [pydicom.dcmread(img_path) for img_path in img_paths]
img_size = np.asarray([[d.Rows, d.Columns] for d in dicom])
new_series_df


,study_id,series_id,instance_number,condition,level,x,y,series_description,fold_id
0,2155667219,2142829,8,Spinal Canal Stenosis,L1/L2,137.317889,90.701131,Sagittal T2/STIR,4
1,2155667219,2142829,8,Spinal Canal Stenosis,L2/L3,128.219343,118.823910,Sagittal T2/STIR,4
2,2155667219,2142829,8,Spinal Canal Stenosis,L3/L4,126.978632,152.323102,Sagittal T2/STIR,4
3,2155667219,2142829,8,Spinal Canal Stenosis,L4/L5,136.077178,175.896607,Sagittal T2/STIR,4
4,2155667219,2142829,8,Spinal Canal Stenosis,L5/S1,149.311427,203.192246,Sagittal T2/STIR,4


In [50]:
keypoints = np.zeros((5, 3), dtype=int)
for row_id in range(len(new_series_df)):
    row = new_series_df.iloc[row_id]
    keypoints[row_id, 0] = int(row.x / img_size[int(row.instance_number), 1] * volume.shape[2])
    keypoints[row_id, 1] = int(row.y / img_size[int(row.instance_number), 0] * volume.shape[1])
    if new_series_df.series_description.iloc[0] == "Axial T2":
        keypoints[row_id, 2] = int(new_series_df.instance_number.iloc[row_id] / len(img_size) * volume.shape[0])
if new_series_df.series_description.iloc[0] != "Axial T2": 
    keypoints[:, 2] = int(new_series_df.instance_number.mean() / len(img_size) * volume.shape[0])
keypoints

array([[137,  90,   4],
       [128, 118,   4],
       [126, 152,   4],
       [136, 175,   4],
       [149, 203,   4]])

In [51]:
label_x = np.zeros((5, volume.shape[2]))
label_y = np.zeros((5, volume.shape[1]))
label_z = np.zeros((5, volume.shape[0]))
for row_id in range(len(new_series_df)):
    row = new_series_df.iloc[row_id]
    x_pos = keypoints[row_id, 0]
    y_pos = keypoints[row_id, 1]
    label_x[row_id, max(int(x_pos)-4, 0):min(int(x_pos)+5, label_x.shape[-1])] = 1.0
    label_y[row_id, max(int(y_pos)-4, 0):min(int(y_pos)+5, label_x.shape[-1])] = 1.0
    z_pos = keypoints[row_id, 2]
    label_z[row_id, z_pos] = 1.0
    if z_pos - 1 > 0:
        label_z[row_id, z_pos - 1] = 1.0
    if z_pos + 1 < label_z.shape[-1]:
        label_z[row_id, z_pos + 1] = 1.0

label_x = torch.tensor(label_x)
label_y = torch.tensor(label_y)
label_z = torch.tensor(label_z)
print(label_x.shape, label_y.shape, label_z.shape)

torch.Size([5, 256]) torch.Size([5, 256]) torch.Size([5, 9])


In [52]:
label_z

tensor([[0., 0., 0., 1., 1., 1., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 0., 0., 0.]], dtype=torch.float64)

In [53]:
label_x[0]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 

In [54]:
KEYPOINT_COLOR = (0, 255, 0) # Green

def add_keypoints(images, keypoints, color=KEYPOINT_COLOR, diameter=10):
    images = images.copy()

    for (x, y, z) in keypoints:
        cv2.circle(images[z], (int(x), int(y)), diameter, (0, 255, 0), -1)
    return images
volume_vis = add_keypoints(volume, keypoints)
fig = plt.figure(figsize=(3, 3))
im = plt.imshow(volume_vis[0])
def draw(i):
    im.set_array(volume_vis[i])
    return [im]
anim = animation.FuncAnimation(
    fig, draw, frames=volume_vis.shape[0], interval=200, blit=True
)
plt.close()
display.HTML(anim.to_jshtml())

In [61]:
# random.seed(7)
transform = A.Compose([
        A.HorizontalFlip(p=1),
        A.ShiftScaleRotate(
            shift_limit=0.0625,
            scale_limit=0.2,
            rotate_limit=15,
            border_mode=cv2.BORDER_CONSTANT, 
            value=0, 
            p=0.9,
        ),
        A.OneOf(
            [
                A.GaussianBlur(p=0.3),
                A.GaussNoise(p=0.3),
                A.PiecewiseAffine(p=0.3),  # IAAPiecewiseAffine
            ],
            p=0.9,
        ),
        A.RandomBrightnessContrast(
            brightness_limit=0.3, contrast_limit=0.3, p=0.3
        ),
        A.RandomResizedCrop(width=256, height=256, scale=(0.4, 1.0), p=0.5),
        ToTensorV2(),
    ],
    keypoint_params=A.KeypointParams(format='xy', label_fields=['class_labels']),
)
class_labels = [0, 1, 2, 3, 4]
transformed = transform(image=volume.transpose((1, 2, 0, 3)).reshape(volume.shape[1], volume.shape[2], -1), keypoints=keypoints[:, :2], class_labels=class_labels)
print(transformed['image'].shape)
volume_vis = transformed['image'].reshape((volume.shape[0], volume.shape[3], volume.shape[1], volume.shape[2])).permute(0, 2, 3, 1).numpy()
print(transformed['keypoints'], transformed['class_labels'])
if len(transformed['keypoints']) > 0:   
    keypoints_vis = np.concatenate([transformed['keypoints'], keypoints[transformed['class_labels'], 2:]], -1).astype(int)
else:
    keypoints_vis = np.asarray([])
volume_vis = add_keypoints(volume_vis, keypoints_vis)
fig = plt.figure(figsize=(3, 3))
im = plt.imshow(volume_vis[0])
def draw(i):
    im.set_array(volume_vis[i])
    return [im]
anim = animation.FuncAnimation(
    fig, draw, frames=volume_vis.shape[0], interval=200, blit=True
)
plt.close()
display.HTML(anim.to_jshtml())

torch.Size([27, 256, 256])
[(133.36526946107784, 26.88), (144.09580838323353, 64.0), (144.09580838323353, 107.52), (127.23353293413174, 136.96), (104.23952095808384, 171.52)] [0, 1, 2, 3, 4]


In [62]:
label_x = np.zeros((5, volume.shape[2]))
label_y = np.zeros((5, volume.shape[1]))
label_z = np.zeros((5, volume.shape[0]))
class_id_map = {i: c for i, c in enumerate(transformed["class_labels"])}
for row_id in range(len(new_series_df)):
    if row_id in transformed["class_labels"]:
        x_pos = keypoints[class_id_map[row_id], 0]
        y_pos = keypoints[class_id_map[row_id], 1]
        label_x[
            row_id,
            max(int(x_pos) - 4, 0) : min(int(x_pos) + 5, label_x.shape[-1]),
        ] = 1.0
        label_y[
            row_id,
            max(int(y_pos) - 4, 0) : min(int(y_pos) + 5, label_x.shape[-1]),
        ] = 1.0
        z_pos = keypoints[class_id_map[row_id], 2]
        label_z[row_id, z_pos] = 1.0
        if z_pos - 1 > 0:
            label_z[row_id, z_pos - 1] = 1.0
        if z_pos + 1 < label_z.shape[-1]:
            label_z[row_id, z_pos + 1] = 1.0

label_x = torch.tensor(label_x)
label_y = torch.tensor(label_y)
label_z = torch.tensor(label_z)

In [63]:
volume.shape

(9, 256, 256, 3)

In [64]:
label_z

tensor([[0., 0., 0., 1., 1., 1., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 0., 0., 0.]], dtype=torch.float64)